In [166]:
#import modules
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from splinter import Browser
from selenium import webdriver

In [167]:
#link to chromedriver.exe
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}

In [358]:
#function that opens browser, parses html, and closes browser
def parse(url):
    with Browser('chrome',**executable_path) as browse:
        browse.visit(url)
        html = browse.html

        return bs(html, 'html.parser')

In [324]:
#html for mars news
marsite = "https://redplanetscience.com/"

marsoup = parse(marsite)

In [446]:
#scrape title and summary of feature news article
titles = marsoup.find_all('div', class_="content_title")
html_news = titles[0]
news_title = html_news.text

html_pgrph = html_news.findNextSibling()
news_pgrph = html_pgrph.text
print(f'{news_title}\n-----------------------\n{news_pgrph}')

From JPL's Mailroom to Mars and Beyond
-----------------------
Bill Allen has thrived as the mechanical systems design lead for three Mars rover missions, but he got his start as a teenager sorting letters for the NASA center.


In [417]:
#html for mars images
featurl = "https://spaceimages-mars.com/"

imgsoup = parse(featurl)

In [419]:
#scrape featureimage
imgs = imgsoup.find_all('img')
srcs = []
for im in imgs:
    srcs.append(im['src'])
urlend = srcs[1]
urlend

'image/featured/mars2.jpg'

In [420]:
#concatenate the image page url with the feature image suffix
featimg_url = featurl + urlend
print(featimg_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


In [239]:
#html for mars fact tables
factsurl = "https://galaxyfacts-mars.com/"

tables = pd.read_html(factsurl)


In [241]:
#grab table 0 with pandas
marscompare = tables[0].set_index(tables[0][0])
marscompare.drop(columns=0,inplace=True)
marscompare.rename(columns = {1: "Mars", 2: "Earth"},inplace=True)
marscompare.drop("Mars - Earth Comparison",inplace=True)
marscompare.index.name = "Mars - Earth Comparison"

In [242]:
#mars-earth comparison table
marscompare

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [430]:
marscomparehtml = marscompare.to_html().replace("\n", '')

In [249]:
#grab table 1 with pandas
marsfcts = tables[1].set_index(tables[1][0])
marsfcts.drop(columns=0,inplace=True)
marsfcts.rename(columns = {1: "Mars"},inplace=True)
marsfcts.index.name = "Mars Facts"

In [250]:
#other mars facts table
marsfcts

,Mars
Mars Facts,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 ( Phobos & Deimos )
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [429]:
marsfctshtml = marsfcts.to_html().replace("\n", '')

In [425]:
#html of hemisphere page
geourl = "https://marshemispheres.com/"
geohtml = parse(geourl)

In [337]:
#scrape the links for the full-size image pages
linkhref = []
for link in geohtml.findAll("a",class_="itemLink product-item"):
    linkhref.append(link["href"])
for href in linkhref:
    i = linkhref.index(href)
    if href in [linkhref[j] for j in range(0,len(linkhref)) if j != i]:
        del linkhref[i]
del linkhref[-1]
linkhref


['cerberus.html', 'schiaparelli.html', 'syrtis.html', 'valles.html']

In [441]:
hrefname = []
for href in linkhref:
    name = href[0:-5]
    hrefname.append(name)
hrefname

['cerberus', 'schiaparelli', 'syrtis', 'valles']

In [442]:
#get html for each full-size image page
htmls = dict.fromkeys(hrefname)
for href in hrefname:
    with Browser('chrome', **executable_path) as browse:
        browse.visit(geourl+href+'.html')
        htmls[href] = bs(browse.html,'html.parser')

In [445]:
#scrape the title and img url from the full-size image pages
hemimglinks = dict.fromkeys(hrefname)
for i in range(4):
    titles = dict.fromkeys(['title','img_url'])
    a = htmls[hrefname[i]].find('a',text='Sample')
    h2 = htmls[hrefname[i]].find('h2', class_="title")
    titles['title'] = h2.text
    titles['img_url'] = geourl+a['href']
    hemimglinks[hrefname[i]] = titles
hemimglinks

{'cerberus': {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 'schiaparelli': {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 'syrtis': {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 'valles': {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}}